---
title: Classifying Palmer's Penguins
author: Will Lavey
date: '2025-02-28'
image: "penguins_cover_photo.jpeg"
description: "The first blog post for CS451, classifying Palmer's Penguins"
format: html
---

# Introduction

Testing blog post functionality.